# Z3 Tutorial

Following examples from https://sat-smt.codes/SAT_SMT_by_example.pdf

In [2]:
from z3 import *

In [4]:
x = Real('x')
y = Real('y')
z = Real('z')
s = Solver()
s.add(3*x + 2*y - z == 1)
s.add(2*x - 2*y + 4*z == -2)
s.add(-x + 0.5*y - z == 0)
s.check(),s.model()

(sat, [y = -2, x = 1, z = -2])

In [6]:
circle, square, triangle = Ints('circle square triangle')
s = Solver()
s.add(circle+circle == 10)
s.add(circle*square+square==12)
s.add(circle*square - triangle*circle == circle)
s.check(),s.model()

(sat, [circle = 5, square = 2, triangle = 1])

In [37]:
# https://xkcd.com/287/

a,b,c,d,e,f= Ints('a b c d e f')
s = Solver()
s.add(a <= 10)
s.add(a >= 0)
s.add(b <= 10)
s.add(b >= 0)
s.add(c <= 10)
s.add(c >= 0)
s.add(d <= 10)
s.add(d >= 0)
s.add(e <= 10)
s.add(e >= 0)
s.add(f <= 10)
s.add(f >= 0)
s.add(a*215+b*275+c*335+d*355+e*420+f*580 == 1505)
#s.add(z3.And(a != 7,b != 1))
s.check()
m=s.model()
m
# while s.check():
#   m = s.model()
#   print(m)
#   s.add(z3.Not(z3.And(a==m["a"], b==m["b"], c==m["c"], d==m["d"], e==m["e"], f==m["f"])))

[d = 0, b = 0, a = 7, f = 0, c = 0, e = 0]

In [45]:
a,b,c,d,e,f = Ints('a b c d e f')
s = Solver()
s.add(215*a + 275*b + 335*c + 355*d + 420*e + 580*f == 1505, a>=0, b>=0, c>=0, d>=0, e>=0, f>=0)

results=[]

# enumerate all possible solutions:
while True:
    if s.check() == sat:
        m = s.model()
        print (m)
        results.append(m)
        block = []
        for d in m:
            c=d()
            block.append(c != m[d])
        s.add(Or(block))
    else:
        print ("results total=", len(results))
        break

[d = 0, b = 0, a = 7, f = 0, c = 0, e = 0]
[c = 0, d = 2, e = 0, f = 1, b = 0, a = 1]
results total= 2


In [46]:
s = Optimize()
workpieces_total=Int('workpieces_total')
cutA,cutB,cutC,cutD = Ints('cutA cutB cutC cutD')
outA,outB=Ints('outA outB')
s.add(cutA>=0)
s.add(cutB>=0)
s.add(cutC>=0)
s.add(cutD>=0)
s.add(outA==cutA*3+cutB*2+cutC*1)
s.add(outB==cutA*1+cutB*6+cutC*9+cutD*13)
s.add(outA==800)
s.add(outB==400)
s.minimize(workpieces_total)
s.check(),s.model()

(sat,
 [cutC = 0,
  cutB = 25,
  cutD = 0,
  outB = 400,
  outA = 800,
  cutA = 250])

In [48]:
s = Solver()
rabbit,cat,dog = Ints('rabbit cat dog')
s.add(cat+rabbit==10)
s.add(dog+rabbit==20)
s.add(dog+cat==24)
s.check(),s.model()

(sat, [rabbit = 3, dog = 17, cat = 7])

In [51]:
# Subset sum

items = [-7,-3,-2,5,8]
vars=[Int(f"var_{i}") for i in range(len(items))]
s = Solver()
rt = []
for i in range(len(items)):
  rt.append(vars[i]*items[i])
  s.add(z3.Or(vars[i]==0,vars[i]==1))

s.add(sum(rt)==0)
s.add(sum(vars)>=1)
s.check(),s.model()

(sat, [var_0 = 0, var_1 = 1, var_2 = 1, var_3 = 1, var_4 = 0])

In [61]:
# The sum of two nonzero real numbers is 4 times their product.
# What is the sum of the reciprocals of the two numbers?

x,y=Reals('x y')
s=Solver()
s.add(x>0,y>0,x+y==4*x*y)
s.check()
m = s.model()
(m,m.evaluate(1/x+1/y))

([y = 1/3, x = 1], 4)

In [66]:
m = BitVec('m', 32)
s = Solver()
divisor=3
const=(0x1234567*divisor)
s.add(const*m==const/divisor)
s.check()
s.model()[m]

2863311531

In [68]:
known=[
"01?10001?",
"01?100011",
"011100000",
"000000000",
"111110011",
"????1001?",
"????3101?",
"?????211?",
"?????????"]

WIDTH=len(known[0])
HEIGHT=len(known)
print ("WIDTH=", WIDTH , "HEIGHT=", HEIGHT)

def chk_bomb(row, col):
  s=Solver()
  cells=[[Int('r%d_c%d' % (r,c)) for c in range(WIDTH+2)] for r in range(HEIGHT+2)]
  # make border
  for c in range(WIDTH+2):
    s.add(cells[0][c]==0)
    s.add(cells[HEIGHT+1][c]==0)
  for r in range(HEIGHT+2):
    s.add(cells[r][0]==0)
    s.add(cells[r][WIDTH+1]==0)
  for r in range(1,HEIGHT+1):
    for c in range(1,WIDTH+1):
      s.add(z3.Or(cells[r][c]==0, cells[r][c]==1))
      t = known[r-1][c-1]
      if t in "012345678":
        s.add(cells[r][c]==0)
        expr = cells[r-1][c-1] + cells[r-1][c] + cells[r-1][c+1] + cells[r][c-1] + cells[r][c+1] + cells[r+1][c-1] + cells[r+1][c] + cells[r +1][c+1]==int(t)
        if False:
          print(expr)
        s.add(expr)

  s.add(cells[row][col]==1)
  if s.check() == unsat:
    print(f"row={row} col={col}, unsat!")

for r in range(1,HEIGHT+1):
  for c in range(1,WIDTH+1):
    if known[r-1][c-1]=="?":
      chk_bomb(r,c)

WIDTH= 9 HEIGHT= 9
row=1 col=3, unsat!
row=6 col=2, unsat!
row=6 col=3, unsat!
row=7 col=4, unsat!
row=7 col=9, unsat!
row=8 col=9, unsat!


In [80]:
def factor(n):
  in1,in2,out=Ints('in1 in2 out')
  s = Solver()
  s.add(out==n, in1*in2==out ,in1>1,in2>1)
  if s.check() == unsat:
    print(f"{n} is prime")
    return [n]
  if s.check() ==unknown:
    print(f"{n} is probably prime")
    return [n]
  m = s.model()
  in1_n=m[in1]
  in2_n=m[in2]
  print(f"factors of {n}: {in1_n}, {in2_n}")
  rt = list(factor(in1_n) + factor(in2_n))
  return rt

factor(123456)


factors of 123456: 24, 5144
factors of 24: 12, 2
factors of 12: 4, 3
factors of 4: 2, 2
2 is prime
2 is prime
3 is prime
2 is prime
factors of 5144: 8, 643
factors of 8: 4, 2
factors of 4: 2, 2
2 is prime
2 is prime
2 is prime
643 is prime


[2, 2, 3, 2, 2, 2, 2, 643]

In [4]:
# Advent of Code 2016 Day 15 https://adventofcode.com/2016/day/15
def Day15Z3():
  def solve(part2=False):
    s = Solver()
    t = Int('t')
    d1Off,d1Size,d1Init = Ints('d1Off d1Size d1Init')
    d2Off,d2Size,d2Init = Ints('d2Off d2Size d2Init')
    d3Off,d3Size,d3Init = Ints('d3Off d3Size d3Init')
    d4Off,d4Size,d4Init = Ints('d4Off d4Size d4Init')
    d5Off,d5Size,d5Init = Ints('d5Off d5Size d5Init')
    d6Off,d6Size,d6Init = Ints('d6Off d6Size d6Init')
    if part2:
      d7Off,d7Size,d7Init = Ints('d7Off d7Size d7Init')
    s.add(d1Off == 1, d1Size == 5, d1Init == 2)
    s.add(d2Off == 2, d2Size == 13, d2Init == 7)
    s.add(d3Off == 3, d3Size == 17, d3Init == 10)
    s.add(d4Off == 4, d4Size == 3, d4Init == 2)
    s.add(d5Off == 5, d5Size == 19, d5Init == 9)
    s.add(d6Off == 6, d6Size == 7, d6Init == 0)
    if part2:
      s.add(d7Off == 7, d7Size == 11, d7Init == 0)
    s.add( (t + d1Off + d1Init) % d1Size == 0 )
    s.add( (t + d2Off + d2Init) % d2Size == 0 )
    s.add( (t + d3Off + d3Init) % d3Size == 0 )
    s.add( (t + d4Off + d4Init) % d4Size == 0 )
    s.add( (t + d5Off + d5Init) % d5Size == 0 )
    s.add( (t + d6Off + d6Init) % d6Size == 0 )
    if part2:
      s.add( (t + d7Off + d7Init) % d7Size == 0 )
    s.add(t > 0)
    s.check()
    model = s.model()
    print(model)
    return model[t]

  return solve(part2=False), # part2 is too slow, take 15 minutes +

Day15Z3()

[d6Off = 6,
 d3Init = 10,
 d1Size = 5,
 d5Off = 5,
 d2Init = 7,
 d6Size = 7,
 d4Off = 4,
 d1Init = 2,
 d5Size = 19,
 d3Off = 3,
 d6Init = 0,
 d4Size = 3,
 d2Off = 2,
 d5Init = 9,
 d3Size = 17,
 d1Off = 1,
 d4Init = 2,
 d2Size = 13,
 t = 148737]


(148737,)